In [ ]:
x_= [0.68, 0.38, 0.46, 0.62, 0.42]


# **Sigmoid**

In [ ]:
def sig(x):
  e = 2.718281828459045 **(-x)
  return 1/(1+e)

In [ ]:
for x in x_:
  print(sig(x))

0.6637386974043527
0.5938731029341427
0.6130141761393355
0.6502185485738271
0.6034832498647262


# **Tanh**

In [ ]:
def tanh(x):
  e =  2.718281828459045 **(-2 * x)
  return (1-e)/(1+e)

In [ ]:
for x in x_:
  print(tanh(x))

0.5915193954318165
0.3627074675780511
0.43008421140197944
0.5511280285381469
0.39693043200507755


# **ReLU**

In [ ]:
def relu(x):
  return max(0,x)

In [ ]:
for x in x_:
  print(relu(x))

0.68
0.38
0.46
0.62
0.42


# **LAB 1**

Importing the required libraries

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import r2_score

Loading the data

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/CPCB.csv')

Exploring the data

In [ ]:
data.head()

,Datetime,NO2,O3,CO,SO2,Temp,RH,PM2.5,PM10
0,01-07-2022 00:00,2.175523,1.171429,233.043584,4.965646,23.336625,72.361834,11.883808,19.943981
1,01-07-2022 01:00,2.175523,1.171429,233.043584,4.965646,23.336625,72.361834,11.883808,19.943981
2,01-07-2022 02:00,2.175523,1.171429,233.043584,4.965646,23.336625,72.361834,11.883808,19.943981
3,01-07-2022 03:00,2.175523,1.171429,233.043584,4.965646,23.336625,72.361834,11.883808,19.943981
4,01-07-2022 04:00,2.175523,1.171429,233.043584,4.965646,23.336625,72.361834,11.883808,19.943981


In [ ]:
data.describe()

,NO2,O3,CO,SO2,Temp,RH,PM2.5,PM10
count,8881.000000,8881.000000,8881.000000,8881.000000,8881.000000,8881.000000,8881.000000,8881.000000
mean,2.176745,1.173476,235.143765,4.978822,23.293260,72.685802,12.092634,20.034472
std,0.585049,2.622279,121.485137,1.170514,2.782650,15.706897,8.870804,15.536308
min,1.765969,0.186224,0.000000,2.920644,17.892000,13.042500,0.279585,0.319974
25%,2.132156,0.447881,165.583675,3.943222,21.000000,68.760000,5.358683,10.096638
50%,2.175523,0.684918,233.043584,4.965646,23.336625,72.361834,11.883808,19.943981
75%,2.241776,1.171429,250.962758,6.037213,25.412000,84.490000,13.989915,21.900803
max,25.885676,34.519615,1332.431136,9.834521,28.319500,99.132500,66.804468,392.741170


In [ ]:
data.shape

(8881, 9)

In [ ]:
data.dtypes

,0
Datetime,object
NO2,float64
O3,float64
CO,float64
SO2,float64
Temp,float64
RH,float64
PM2.5,float64
PM10,float64


In [ ]:
data.columns

Index(['Datetime', 'NO2', 'O3', 'CO', 'SO2', 'Temp', 'RH', 'PM2.5', 'PM10'], dtype='object')

Pre-processing the data

In [ ]:
data.isnull().sum()
# No missing values

,0
Datetime,0
NO2,0
O3,0
CO,0
SO2,0
Temp,0
RH,0
PM2.5,0
PM10,0


In [ ]:
data_numerical = data.select_dtypes(include=np.number)

In [ ]:
data_selected = data_numerical[['CO', 'SO2', 'PM10', 'PM2.5']]

minmax = MinMaxScaler()
data = minmax.fit_transform(data_selected)

In [ ]:
numerical_data = data[['NO2', 'O3', 'CO', 'SO2', 'Temp', 'RH', 'PM2.5', 'PM10']]

Q1 = np.percentile(numerical_data, 25, axis=0)
Q3 = np.percentile(numerical_data, 75, axis=0)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

lower_df = pd.DataFrame([lower_bound], columns=numerical_data.columns)
upper_df = pd.DataFrame([upper_bound], columns=numerical_data.columns)

# Winsorize: clip values outside bounds
data_winsorized = numerical_data.clip(lower=lower_df, upper=upper_df, axis=1)

data[['NO2', 'O3', 'CO', 'SO2', 'Temp', 'RH', 'PM2.5', 'PM10']] = data_winsorized

NameError: name 'df' is not defined

ANN

In [ ]:
column_names = [ 'CO', 'SO2', 'PM10','PM2.5']
df = pd.DataFrame(data, columns=column_names)

X = df.drop('PM2.5', axis=1)
y = df['PM2.5']

X.columns = ['CO', 'SO2', 'PM10']
df = pd.DataFrame(X, columns=X.columns)
df['target'] = y
print(df.head())
print(df.head())

         CO       SO2      PM10    target
0  0.174901  0.295782  0.050008  0.174434
1  0.174901  0.295782  0.050008  0.174434
2  0.174901  0.295782  0.050008  0.174434
3  0.174901  0.295782  0.050008  0.174434
4  0.174901  0.295782  0.050008  0.174434
         CO       SO2      PM10    target
0  0.174901  0.295782  0.050008  0.174434
1  0.174901  0.295782  0.050008  0.174434
2  0.174901  0.295782  0.050008  0.174434
3  0.174901  0.295782  0.050008  0.174434
4  0.174901  0.295782  0.050008  0.174434


In [ ]:
X_train, X_test,y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
print("Before Scaling")
print(X_train)
print(X_test)

Before Scaling
            CO       SO2      PM10
3872  0.176699  0.081440  0.010462
1345  0.087379  0.079397  0.016574
7978  0.141748  0.293252  0.028333
5924  0.161165  0.518849  0.056394
7301  0.077670  0.185516  0.013981
...        ...       ...       ...
5734  0.244660  0.548549  0.081525
5191  0.054369  0.410774  0.011652
5390  0.174901  0.295782  0.050008
860   0.174901  0.295782  0.050008
7270  0.141748  0.639946  0.053109

[7104 rows x 3 columns]
            CO       SO2      PM10
7810  0.128155  0.344566  0.036936
736   0.079612  0.056251  0.014527
4423  0.172816  0.227895  0.062688
2138  0.062136  0.126202  0.014277
3685  0.174901  0.295782  0.050008
...        ...       ...       ...
8377  0.174901  0.295782  0.050008
2580  0.122330  0.142796  0.026788
6149  0.277670  0.602502  0.112256
7963  0.227184  0.302442  0.033413
2287  0.174901  0.295782  0.050008

[1777 rows x 3 columns]


In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print("After Scaling")
print(X_train)
print(X_test)

After Scaling
[[ 0.00419242 -1.27363307 -1.0123011 ]
 [-0.98032254 -1.28570194 -0.85649354]
 [-0.38105256 -0.02199069 -0.55677897]
 ...
 [-0.01562559 -0.00703685 -0.00429756]
 [-0.01562559 -0.00703685 -0.00429756]
 [-0.38105256  2.02669998  0.07476478]]
[[-0.53087006  0.28123967 -0.33748035]
 [-1.06593253 -1.42248247 -0.90867975]
 [-0.03861258 -0.40819453  0.3189263 ]
 ...
 [ 1.11712238  1.80543736  1.58240228]
 [ 0.5606574   0.03231922 -0.42728466]
 [-0.01562559 -0.00703685 -0.00429756]]


In [ ]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='relu')  # no activation for regression
])

In [ ]:
# Compile and train
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.1)

Epoch 1/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.0143 - mae: 0.0797 - val_loss: 0.0037 - val_mae: 0.0355
Epoch 2/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0027 - mae: 0.0304 - val_loss: 0.0023 - val_mae: 0.0284
Epoch 3/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0021 - mae: 0.0269 - val_loss: 0.0021 - val_mae: 0.0275
Epoch 4/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0020 - mae: 0.0265 - val_loss: 0.0021 - val_mae: 0.0268
Epoch 5/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0018 - mae: 0.0251 - val_loss: 0.0021 - val_mae: 0.0269
Epoch 6/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0019 - mae: 0.0255 - val_loss: 0.0021 - val_mae: 0.0280
Epoch 7/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0018 - mae: 0.0247 - val_loss: 0.0020 - val_mae: 0.0262
Epoch 8/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0017 - mae: 0.0244 - val_loss: 0.0020 - val_mae: 0.0264
Epoch 9/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step -

In [ ]:
# Evaluate
loss, mae = model.evaluate(X_test, y_test)
print(f"Test Loss (MSE): {loss:.4f}, Test MAE: {mae:.4f}")

56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0014 - mae: 0.0229
Test Loss (MSE): 0.0015, Test MAE: 0.0234


In [ ]:
# Predict using the model
y_pred = model.predict(X_test)

# Compute R² Score
r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2:.4f}")


56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
R² Score: 0.9128
